In [14]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/fb.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_essays))

9917


In [15]:
data_essays.drop(data_essays.head(1).index, inplace=True)

In [16]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from empath import Empath
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

accuracy = 0.0
rmse = 0.0
h_loss = 0.0
for i in range(10):
    rand_essays = []
    temp_data = data_essays.copy()
    rand_essays = random.sample(range(1,len(temp_data)-1),1000)
    test = temp_data.iloc[rand_essays].copy()
    train = temp_data.drop(rand_essays)
    train.drop(train.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    test.drop(test.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    
    corpus = []
    truth = []
    for row in train.iterrows():
        Y = [0,0,0,0,0]
        corpus.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth.append(Y)
    truth = np.array(truth)    
    lexicon = Empath()
    feat = []
    a = {}
    for row in corpus:
        a = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a.items():
            temp.append(v)
        feat.append(temp)
    feat = np.array(feat)
    feat_norm = normalize(feat, axis=1, norm='l1')
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(feat_norm, truth)
    
    test_data = []
    truth_test = []
    for row in test.iterrows():
        Y = [0,0,0,0,0]
        test_data.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth_test.append(Y)
    truth_test = np.array(truth_test)    
    test_feat = []
    a_test = {}
    for row in test_data:
        a_test = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a_test.items():
            temp.append(v)
        test_feat.append(temp)
    test_feat = np.array(test_feat)
    test_feat_norm = normalize(test_feat, axis=1, norm='l1')
    output = clf.predict(test_feat_norm)
    err = [sqrt(mean_squared_error(truth_test[:,i], output[:,i])) for i in range(len(truth_test[0,:]))] 
    acc = accuracy_score(truth_test, output, normalize=True)
    loss = hamming_loss(truth_test, output)
    accuracy += acc
    #rmse += err
    h_loss += loss
    print("Accuracy for interation " + str(i) + ": " + str(acc))
    print("Hamming Loss for interation " + str(i) + ": " + str(loss))
    for each in err:
        print("RMSE for interation " + str(i) + ": " + str(each))
    del test
    del train
    del temp_data
print("-------------------------------")
print("Average accuracy is :" + str(accuracy/10))
print("Average haming loss is :" + str(h_loss/10))
#print("Average RMSE loss is :" + str(rmse/10))

Accuracy for interation 0: 0.472
Hamming Loss for interation 0: 0.2252
RMSE for interation 0: 0.4745524207081869
Accuracy for interation 1: 0.487
Hamming Loss for interation 1: 0.2152
RMSE for interation 1: 0.4638965401897281
Accuracy for interation 2: 0.487
Hamming Loss for interation 2: 0.2084
RMSE for interation 2: 0.45650848842053304
Accuracy for interation 3: 0.5
Hamming Loss for interation 3: 0.2156
RMSE for interation 3: 0.46432747064975594
Accuracy for interation 4: 0.498
Hamming Loss for interation 4: 0.2082
RMSE for interation 4: 0.45628938186199336
Accuracy for interation 5: 0.485
Hamming Loss for interation 5: 0.213
RMSE for interation 5: 0.46151923036857306
Accuracy for interation 6: 0.491
Hamming Loss for interation 6: 0.2104
RMSE for interation 6: 0.4586937976471886
Accuracy for interation 7: 0.483
Hamming Loss for interation 7: 0.2192
RMSE for interation 7: 0.468187996428785
Accuracy for interation 8: 0.496
Hamming Loss for interation 8: 0.211
RMSE for interation 8: 0.4